In [ ]:
import cv2
import numpy as np
import matplotlib.pylab as plt
from ipynb.fs.full.ColourSpaceConversionUtil import *

def maintainAspectRatioWhileResizing(originalImage, width, height, pixelAreaRelation):
    
    # The dimensions of the image are initialized and the shape of the image is determined
    imageDimension = None
    (h, w) = originalImage.shape[:2]

    #The original image is returned if the resizing width and height are not provided at the time of invoking
    if width is None and height is None:
        return originalImage
    
    if width is None:
        #Calculate the aspect ratio of the resized height to the original height if the resized width is none (or not provided)
        ratio = height / float(h)
        #Calculate the resized image dimension
        imageDimension = (int(w * ratio), height)
    else:
        #Calculate the aspect ratio of the resized width to the original width if the resized height is none (or not provided)
        ratio = width / float(w)
        #Calculate the resized image dimension
        imageDimension = (width, int(h * ratio))

    #The original image is now resized and returned based on the calculated image dimensions while maintaining the aspect ratio
    return cv2.resize(originalImage, imageDimension, interpolation=pixelAreaRelation)


def probabilisticHoughLineTransform(rgbImage):
    
    #Purpose: Grayscale image is preferred over a colour image in Hough Transform
    grayscale_image=colourRGBToGrayConversion(rgbImage)
    
    
    #Edge Detection using Canny Edge Detector
    #Arg1 is the grayscale image 
    #Arg2 is the first threshold
    #Arg3 is the second threshold
    #Arg4 is the aperture size
    edges = cv2.Canny(grayscale_image, 40, 55, apertureSize=3)
    
    #Mapping the edge points to the Hough space using the Probabilistic Hough Line transformation algorithm that is available in OpenCV
    #Arg1 is the edge detected image 
    #Arg2 is the rho value which is the distance resolution of the accumulator (in pixels)
    #Arg3 is the theta value which is the angle resolution of the accumulator (in radians)
    #Arg4 is the accumulator's threshold parameter which implies that lines whose value is greater than the mentioned threshold value are only returned
    #Arg5 is the minimum length of the detected line. Lines less than the minimum length shall be ignored.
    #Arg6 is the maximum permissible gap between the line segments so that each line segment is considered as a single line
    lines = cv2.HoughLinesP(edges,1,np.pi/180,180,minLineLength=50,maxLineGap=50)
    
    #Each line is represented by two points - P1(x1,y1) and P2(x2,y2)
    for line in lines:
        x1,y1,x2,y2 = line[0]
        
        #Arg1 is the color image
        #Arg2 is the Cartesian co-ordinate of point 1 i.e., P1
        #Arg3 is the Cartesian co-ordinate of point 2 i.e., P2
        #Arg4 is the colour (i.e., RGB value)
        #Arg5 is the thickness of the line
        cv2.line(rgbImage,(x1,y1),(x2,y2),(0,255,0),2)
    
    return edges


def region_of_interest(image, roiVertices):
    
    # Creating a mask using numpy
    mask = np.zeros_like(image)
    
    # Setting mask colour
    matchMaskColor = 255
    
    # Applying the mask and the region of interest vertices to the image
    cv2.fillPoly(mask, roiVertices, matchMaskColor)
    
    # Performing bitwise AND to obtain the masked image
    maskedImage = cv2.bitwise_and(image, mask)
    
    return maskedImage